In [8]:
import numpy as np
import time

from collections import namedtuple

In [2]:
'''
struct to storage order price and volume to trade

Usage:
a = Order(price=1, volume=1)
'''

Order = namedtuple('order', 'id price volume')

In [3]:
class dynamic_order:
    '''
    extension of Order, used in making deals
    '''
    def __init__(self, order):
        assert isinstance(order, Order)
        self.id = order.id
        self.price = order.price
        self.volume = order.volume
    @property
    def show(self):
        print('id : %d' % self.id)
        print('price : %d' % self.price)
        print('volume : %d' % self.volume)

In [4]:
class Trader:
    '''
    A primary class to define an investor
    '''
    def __init__(self, id):
        self.id = id
        self.cash = 0 # 现金
        self.cash_history = []
        self.asset = 0 # 总资产
        self.asset_history = []
        self.stock = 0 # 持股
        self.stock_history = []
        self.trade_history = [] # 交易记录
        self.price = 0
        self.order = Order(id=self.id,
                           price=0,
                           volume=0)
        
    def gen_order(self):
        self.order = Order(id=self.id,
                           price=np.random.randint(-10, 10), 
                           volume=np.random.randint(1, 5))
        
    def update(self, price, vol):
        vol *= np.sign(self.order.price)
        price = abs(price)
        self.cash -= price * vol
        self.cash_history.append(self.cash)
        self.stock += vol
        self.stock_history.append(self.stock)
        self.trade_history.append((price, vol))
        self.price = price
        self.asset = self.cash + self.stock * self.price
        self.asset_history.append(self.asset)
        
    @property
    def show(self):
        print('id : %d, cash : %d, stock : %d, stock_price : %d, asset : %d' % 
             (self.id, self.cash, self.stock, self.price, self.asset))
        
#         print('asset : %d' % self.asset)
#         print('stock : %d' % self.stock)
        

In [5]:
def make_deals(orders):
    buy_list = [dynamic_order(order) for order in orders if order.price>0]
    sell_list = [dynamic_order(order) for order in orders if order.price<0]

    buy_list = sorted(buy_list, key=lambda x:(-x.price, x.id))
    sell_list = sorted(sell_list, key=lambda x:(abs(x.price), x.id))
    deal_result = np.zeros(len(orders), int)
    price = 0
    if len(buy_list)==0 or len(sell_list)==0:
        return price, deal_result
    while sell_list[0].price+buy_list[0].price>=0:
        if sell_list[0].volume>=buy_list[0].volume:
            sell_list[0].volume -= buy_list[0].volume
            deal_result[buy_list[0].id] += buy_list[0].volume
            deal_result[sell_list[0].id] += buy_list[0].volume
            if sell_list[0].volume==buy_list[0].volume:
                price = (buy_list[0].price + abs(sell_list[0].price)) / 2
            else:
                price = abs(sell_list[0].price)
            _ = buy_list.pop(0)
        else:
            buy_list[0].volume -= sell_list[0].volume
            deal_result[buy_list[0].id] += sell_list[0].volume
            deal_result[sell_list[0].id] += sell_list[0].volume
            price = buy_list[0].price
            _ = sell_list.pop(0)           
        price = abs(price)
        if len(buy_list)==0 or len(sell_list)==0:
            break
    return price, deal_result

In [6]:
investors = [Trader(i) for i in range(200)]

In [9]:
start = time.time()
for step in range(3000):
    orders = []
    for investor in investors:
        investor.gen_order()
        orders.append(investor.order)

    price, deal_result = make_deals(orders)
    for i,investor in enumerate(investors):
        investor.update(price=price, vol=deal_result[i])
        
print('用时:', time.time()-start)

用时: 5.2292797565460205


In [10]:
len(orders)

200

In [11]:
for investor in investors:
    investor.show

id : 0, cash : -828, stock : 176, stock_price : 5, asset : 52
id : 1, cash : 237, stock : -27, stock_price : 5, asset : 102
id : 2, cash : -254, stock : 60, stock_price : 5, asset : 46
id : 3, cash : 218, stock : -39, stock_price : 5, asset : 23
id : 4, cash : -371, stock : 74, stock_price : 5, asset : -1
id : 5, cash : -529, stock : 96, stock_price : 5, asset : -49
id : 6, cash : 207, stock : -14, stock_price : 5, asset : 137
id : 7, cash : 153, stock : -28, stock_price : 5, asset : 13
id : 8, cash : -572, stock : 126, stock_price : 5, asset : 58
id : 9, cash : -1037, stock : 211, stock_price : 5, asset : 18
id : 10, cash : 364, stock : -77, stock_price : 5, asset : -21
id : 11, cash : -283, stock : 56, stock_price : 5, asset : -3
id : 12, cash : -356, stock : 82, stock_price : 5, asset : 54
id : 13, cash : -372, stock : 84, stock_price : 5, asset : 48
id : 14, cash : -650, stock : 129, stock_price : 5, asset : -5
id : 15, cash : -868, stock : 185, stock_price : 5, asset : 57
id : 16,

In [80]:
import time
a = time.time()